In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/a/a270230/LKF/lkf_detection/functions/')
from statistics_functions import *
sys.path.insert(1, '/home/a/a270230/LKF/lkf_tools/lkf_tools/')
from dataset import *
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
res = '4km'

path = '/work/bk1377/a270230/'
path_stat = path + 'statistics/'
path_ds = path + f'datasets/{res}/'

In [3]:
# load mean ice concentration, total ice covered area, mean ice thickness,
# and total ice volume for all years of the model run (1986 - 2100 for 4km,
# 2013 - 2020 & 2093 - 2100 for 1km)
a_mean, area_total, h_mean, ice_vol_total, years_all = np.load(
    path_stat + f'a_mean_tot_h_mean_tot_{res}.npy', allow_pickle=True)

# make an array of the right np.shape out of area_total
arr = np.zeros((len(area_total),365))
for year, area_total_year in enumerate(area_total):
    arr[year,:] = area_total_year

area_total = arr

In [4]:
# select the years you want to analyze
years = [i for i in range(2013,2021)]
years += [i for i in range(2093,2101)]

inds = [np.where(years_all==year)[0][0] for year in years]
area_total = area_total[inds]

In [5]:
# load the data from the lkf_data files
files = [path_ds + f'ds_{year}_{res}.npy' for year in years]
datasets, lkfs = get_lkfs(files)

# load tracks and paths (paths_all also includes the paths
# that are going through each timestep)
tracks = get_tracks(datasets)
paths, paths_all = get_paths(lkfs, tracks)

# calculate horizontal spatial resolution (same for all datasets/ years)
res_km = get_res_km(datasets[0])

np.save(path_stat + f'lkfs_paths_{res}.npy', np.array([years, lkfs, paths, paths_all], dtype='object'))

In [6]:
# calculate metrics
n_lkfs = get_n_lkfs(lkfs)
rho_lkfs = n_lkfs / area_total * 10000
length, mean_length, total_length = get_lkf_length(lkfs, res_km)
lifetimes, mean_lifetime = get_lkf_lifetimes(paths)
# lifetimes_all includes the lifetimes of LKFs that are
# already counted in previous timesteps
lifetimes_all, _ = get_lkf_lifetimes(paths_all)

In [7]:
# create lkf dictionary only if it does not already exist
try: LKFs = np.load(path_stat + f'LKFs_{res}.npy', allow_pickle=True)[0]
except: LKFs = dict()

# calculate decadal mean and standart deviation of each lkf variable
# and store it in the dictionary
for ystart, yend in zip([2013, 2093], [2020, 2100]):
    
    df = pd.DataFrame()
    df['number av'], df['number sd']               = av_sd(n_lkfs, ystart, yend, years)
    df['density av'], df['density sd']             = av_sd(rho_lkfs, ystart, yend, years)
    df['mean length av'], df['mean length sd']     = av_sd(mean_length, ystart, yend, years)
    df['total length av'], df['total length sd']   = av_sd(total_length, ystart, yend, years)
    df['mean lifetime av'], df['mean lifetime sd'] = av_sd(mean_lifetime, ystart, yend, years)

    decade = f'{ystart} - {yend}'
    LKFs[decade] = df

# also store each lkf variable for each year (without averaging)
for y, year in enumerate(years):
    df_y = pd.DataFrame(n_lkfs[y], columns=['number'])
    df_y['density']       = rho_lkfs[y]
    df_y['mean length']   = mean_length[y]
    df_y['total length']  = total_length[y]
    df_y['mean lifetime'] = mean_lifetime[y]
    
    LKFs[f'y{year}'] = dict(
        total = df_y,
    )
    
    # store variables of individual lkfs
    for d in range(365):
        df_d = pd.DataFrame(length[y][d], columns=['length'])
        df_d['lifetime'] = lifetimes_all[y][d]
        
        LKFs[f'y{year}'][f'd{d}'] = df_d
        
np.save(path_stat + f'LKFs_{res}', [LKFs])